In [1]:
from sampo.schemas import WorkGraph, WorkUnit, uuid_str, WorkerReq, Time
from sampo.generator import SimpleSynthetic, SyntheticGraphType
from random import Random
from sampo.schemas.stochastic_graph import ProbabilisticFollowingStochasticGraphScheme
from sampo.generator.environment import get_contractor_by_wg
from sampo.pipeline import SchedulingPipeline
from sampo.scheduler import HEFTScheduler
from sampo.schemas import WorkerProductivityMode, IntervalGaussian
from sampo.schemas.time_estimator import DefaultWorkEstimator, WorkTimeEstimator
from sampo.scheduler.utils.time_computaion import work_priority, calculate_working_time_cascade
from sampo.scheduler.heft.prioritization import prioritization_nodes, stochastic_prioritization, ford_bellman
from sampo.schemas import WorkGraph, GraphNode

Can not find native module; switching to default


In [2]:
rand = Random(123)
ss = SimpleSynthetic(rand)

def get_checks_wg(nodes_count: int) -> WorkGraph:
    nodes = [GraphNode(work_unit=WorkUnit(id=uuid_str(rand),
                                          name=f'Check #{i}',
                                          worker_reqs=[WorkerReq(kind='engineer', volume=Time(0), min_count=1, max_count=3)]),
                       parent_works=[]) for i in range(nodes_count)]
    return WorkGraph.from_nodes(nodes)

fixed_wg = get_checks_wg(100)
fixed_wg.vertex_count

102

In [3]:
# construct the stochastic graph scheme
graph_scheme = ProbabilisticFollowingStochasticGraphScheme(rand=rand, wg=fixed_wg)
defect_graphs = []
for i, node in enumerate(fixed_wg.nodes):
    defect = ss.graph_nodes(top_border=20)
    defect_graphs.append(defect)
    
    defect_prob = rand.random() * 0.5
    graph_scheme.add_part(node=node.id, nodes=defect, prob=defect_prob)

scheme = graph_scheme.prepare_graph()
perfect_wg = graph_scheme.prepare_graph().to_work_graph()
print(perfect_wg.vertex_count)

294


In [4]:
scheme.to_work_graph().vertex_count

310

In [5]:
def construct_work_estimator(i: int, productivity: WorkerProductivityMode) -> WorkTimeEstimator:
    work_estimator = DefaultWorkEstimator()
    work_estimator.set_productivity_mode(productivity)
    for worker in ['driver', 'fitter', 'manager', 'handyman', 'electrician', 'engineer']:
        work_estimator.set_worker_productivity(IntervalGaussian(0.2 * i + 0.2, 1, 0, 2), worker)
    return work_estimator

work_estimator = construct_work_estimator(5, WorkerProductivityMode.Static)

In [6]:
from sampo.scheduler.heft.prioritization import prioritization

static_prioritization = prioritization(fixed_wg, work_estimator)
static_prioritization

[7813931f-10a7-42a7-9a2b-40e5a47540d1,
 0186a2cf-a281-1f70-9c18-a25020487f69,
 01b0366f-11e3-74ee-60aa-0d3098fc4743,
 0299536a-0b4c-1837-7617-7fa8c06c250e,
 042cc7c2-560b-0cb9-c75b-693192bb2bfd,
 04649df7-e40c-0a13-e41c-e352fec9b017,
 0546d9e6-ab1f-5a09-7469-e0f122d1ee44,
 0560d4e0-3174-0bd3-74d2-b828227d8219,
 0874bac6-78f6-76de-f4ad-162a4335d28a,
 0b03aa4d-65ee-7b24-43b9-74fc527257ae,
 15700b61-5128-c2aa-99a0-46d315c91d91,
 1a1f141b-72c5-3633-bb66-ecb350c1eb44,
 1e9be463-8460-e4f5-5675-19d1f994fbe7,
 222aef26-89a5-8715-8df5-fc8e57553ab4,
 2293202c-28df-a298-0d49-6d4bda22594f,
 275cecdc-2d76-8558-fb0d-7d657bdb11b4,
 2898878d-073b-eb3c-c63a-1434ca862126,
 3772b9f8-5bd1-d560-50c5-0acb6bb7097e,
 3b1e1977-908b-3b0a-44ac-04d231b4af8d,
 49a6dbed-a388-73b0-0f4a-38e6e8be44f2,
 4e1ad87e-c4cf-bb8a-0968-02f6d67c8720,
 4e60bf9b-899b-fbfc-6320-1a1584c944fc,
 528c9ae7-fc3f-c73c-0775-8380677a20a7,
 55bc0a69-2b8d-6f81-430b-0b04a8778c53,
 573e8d77-550a-e889-8ff4-1e8d8944897c,
 57e8b4a2-7e95-e03c-f868-

In [7]:
stochastic_wg = graph_scheme.prepare_graph().to_work_graph()
stochastic_wg.vertex_count

318

In [8]:
def stochastic_prioritization_function(wg: WorkGraph, work_estimator: WorkTimeEstimator) -> list[GraphNode]:
    # wg is stochastic-generated graph
    # here we can use only information from static graph
    # let's use static prioritization

    order = []
    for original_static_node in static_prioritization:
        static_node = stochastic_wg[original_static_node.id]
        order.append(static_node)
        # haha, get all defects...
        children = static_node.get_following_nodes()
        children_prioritization = prioritization_nodes(children, work_estimator)
        order.extend(children_prioritization)
    return order


def stochastic_prioritization_function_enhanced(wg: WorkGraph, work_estimator: WorkTimeEstimator) -> list[GraphNode]:
    # wg is stochastic-generated graph
    # here we can use information from static graph's followers statistics
    # let's use static prioritization

    weights = {node: -(work_priority(node, calculate_working_time_cascade, work_estimator) + scheme.average_labor_cost(node)) # stochastic_wg[node.id].get_following_nodes())
               for node in static_prioritization}

    path_weights = ford_bellman(static_prioritization, weights)

    ordered_nodes = [i[0] for i in sorted(path_weights.items(), key=lambda x: (x[1], x[0].id))
                     if not i[0].is_inseparable_son()]
    final_order = []
    for original_static_node in ordered_nodes:
        static_node = stochastic_wg[original_static_node.id]
        final_order.append(static_node)
        children_prioritization = prioritization_nodes(static_node.get_following_nodes(), work_estimator)
        final_order.extend(children_prioritization)
    return final_order

In [9]:
contractors = [get_contractor_by_wg(stochastic_wg)]

In [10]:
scheduler = HEFTScheduler(stochastic_prioritization_f=stochastic_prioritization, work_estimator=work_estimator)

project1 = SchedulingPipeline.create() \
    .wg(stochastic_wg) \
    .contractors(contractors) \
    .work_estimator(work_estimator) \
    .schedule(scheduler) \
    .finish()[0]

project1.schedule.execution_time

138

In [11]:
project1.schedule.full_schedule_df

,idx,task_id,task_name,task_name_mapped,contractor,cost,volume,measurement,start,finish,duration,workers,scheduled_work_object
0,0,f5e77d4d-cf30-cf2d-9097-a70286ee6241,Check #17,Check #17,bebb2e9d-7bc0-4e29-8a15-08f8866bb760,0,0.0,unit,0,0,0,{'engineer': 2},ScheduledWork[work_unit=f5e77d4d-cf30-cf2d-909...
1,1,610e4ad5-09c4-7055-dff4-948fe6b4f832,Check #2,Check #2,bebb2e9d-7bc0-4e29-8a15-08f8866bb760,0,0.0,unit,0,0,0,{'engineer': 2},ScheduledWork[work_unit=610e4ad5-09c4-7055-dff...
2,2,7ca5d0ed-e9e4-b2dd-1525-e9d3d1f7fbe4,Check #25,Check #25,bebb2e9d-7bc0-4e29-8a15-08f8866bb760,0,0.0,unit,0,0,0,{'engineer': 2},ScheduledWork[work_unit=7ca5d0ed-e9e4-b2dd-152...
3,3,573e8d77-550a-e889-8ff4-1e8d8944897c,Check #3,Check #3,bebb2e9d-7bc0-4e29-8a15-08f8866bb760,0,0.0,unit,0,0,0,{'engineer': 2},ScheduledWork[work_unit=573e8d77-550a-e889-8ff...
4,4,d4e3fda5-1691-79e2-9611-516c30665bc3,Check #30,Check #30,bebb2e9d-7bc0-4e29-8a15-08f8866bb760,0,0.0,unit,0,0,0,{'engineer': 2},ScheduledWork[work_unit=d4e3fda5-1691-79e2-961...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,313,b214b68a-123e-8c9c-177b-069d4bd7b29c,start of project,start of project,bebb2e9d-7bc0-4e29-8a15-08f8866bb760,0,0.0,unit,137,137,0,{},ScheduledWork[work_unit=b214b68a-123e-8c9c-177...
314,314,865b88c4-964b-7d51-323c-c0ab76fb4499,start of project,start of project,bebb2e9d-7bc0-4e29-8a15-08f8866bb760,0,0.0,unit,137,137,0,{},ScheduledWork[work_unit=865b88c4-964b-7d51-323...
315,315,f777fae8-eebb-369e-8af3-23e6cafb5478,start of project,start of project,bebb2e9d-7bc0-4e29-8a15-08f8866bb760,0,0.0,unit,137,137,0,{},ScheduledWork[work_unit=f777fae8-eebb-369e-8af...
316,316,66498021-b0d7-23a0-cdd4-d0d5ad0e7487,start of project,start of project,bebb2e9d-7bc0-4e29-8a15-08f8866bb760,0,0.0,unit,137,137,0,{},ScheduledWork[work_unit=66498021-b0d7-23a0-cdd...


In [12]:
from sampo.scheduler.heft.prioritization import stochastic_prioritization

scheduler = HEFTScheduler(prioritization_f=stochastic_prioritization_function_enhanced, work_estimator=work_estimator)

project2 = SchedulingPipeline.create() \
    .wg(stochastic_wg) \
    .contractors(contractors) \
    .work_estimator(work_estimator) \
    .schedule(scheduler) \
    .finish()[0]

project2.schedule.execution_time

109

In [31]:
project2.schedule.full_schedule_df

,idx,task_id,task_name,task_name_mapped,contractor,cost,volume,measurement,start,finish,duration,workers,scheduled_work_object
0,0,6364d5e0-d65a-2a5e-70e1-ad82c4710bb7,minimal road,minimal road,646f260e-3de7-4b41-8e23-3afc5a96db56,1770,6.009262,km,0,1,1,"{'driver': 53, 'manager': 17, 'handyman': 107}",ScheduledWork[work_unit=6364d5e0-d65a-2a5e-70e...
1,1,fc62789c-12a7-567c-652b-54d3aa7f5864,start of project,start of project,646f260e-3de7-4b41-8e23-3afc5a96db56,0,0.000000,unit,0,0,0,{},ScheduledWork[work_unit=fc62789c-12a7-567c-652...
2,2,ac7c2e69-7f2e-eb7e-0352-05d79b683076,start of project,start of project,646f260e-3de7-4b41-8e23-3afc5a96db56,0,0.000000,unit,0,0,0,{},ScheduledWork[work_unit=ac7c2e69-7f2e-eb7e-035...
3,3,e5182800-e282-110b-eda7-ffc2d14f568f,temporary road,temporary road,646f260e-3de7-4b41-8e23-3afc5a96db56,1770,6.009262,km,1,2,1,"{'driver': 53, 'manager': 17, 'handyman': 107}",ScheduledWork[work_unit=e5182800-e282-110b-eda...
4,4,2e27cdd5-ac91-0a39-f166-f41b5bd840c6,final road,final road,646f260e-3de7-4b41-8e23-3afc5a96db56,3540,6.009262,km,2,4,2,"{'driver': 53, 'manager': 17, 'handyman': 107}",ScheduledWork[work_unit=2e27cdd5-ac91-0a39-f16...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,349,0d68b38b-dcab-bde0-426c-8b90656c335b,start of project,start of project,646f260e-3de7-4b41-8e23-3afc5a96db56,0,0.000000,unit,704,704,0,{},ScheduledWork[work_unit=0d68b38b-dcab-bde0-426...
350,350,1273ec81-d2fe-a0e5-c57a-888052265e5b,start of project,start of project,646f260e-3de7-4b41-8e23-3afc5a96db56,0,0.000000,unit,704,704,0,{},ScheduledWork[work_unit=1273ec81-d2fe-a0e5-c57...
351,351,42815fb5-b334-80a3-7137-803fe4b0b71f,start of project,start of project,646f260e-3de7-4b41-8e23-3afc5a96db56,0,0.000000,unit,704,704,0,{},ScheduledWork[work_unit=42815fb5-b334-80a3-713...
352,352,62390465-7ca4-ff07-dfce-4c513751593c,start of project,start of project,646f260e-3de7-4b41-8e23-3afc5a96db56,0,0.000000,unit,704,704,0,{},ScheduledWork[work_unit=62390465-7ca4-ff07-dfc...


In [38]:
s = set(project1.schedule.full_schedule_df.task_id).difference(project2.schedule.full_schedule_df.task_id)
s

{'562a77eb-098e-7013-7df0-71e73210e250',
 '649f5874-02c0-f38b-9ea9-dc08f8206c13'}

In [40]:
import pandas as pd

project1.schedule.full_schedule_df[project1.schedule.full_schedule_df.task_id.isin(s)]

,idx,task_id,task_name,task_name_mapped,contractor,cost,volume,measurement,start,finish,duration,workers,scheduled_work_object
0,0,649f5874-02c0-f38b-9ea9-dc08f8206c13,start of project,start of project,646f260e-3de7-4b41-8e23-3afc5a96db56,0,0.0,unit,0,0,0,{},ScheduledWork[work_unit=649f5874-02c0-f38b-9ea...
355,355,562a77eb-098e-7013-7df0-71e73210e250,finish of project,finish of project,646f260e-3de7-4b41-8e23-3afc5a96db56,0,0.0,unit,785,785,0,{},ScheduledWork[work_unit=562a77eb-098e-7013-7df...


In [13]:
# [node for lst in self._node2followers.values() for lst2, _ in lst for node in lst2 if node.work_unit.name == 'finish of project']
# [(g1, g2) for g1 in defect_graphs for g2 in defect_graphs if len(set(g1).intersection(set(g2))) > 0]
# sorted([node for lst in self._node2followers.values() for lst2, _ in lst for node in lst2], key=lambda x: x.id)
# sorted([(node, prob) for lst in self._node2followers.values() for lst2, prob in lst for node in lst2], key=lambda x: x[0].id)